# Day 22 Monkey Map

Urghr.. back to maps. I don't enjoy these so much. 

Looks like LOGO... rotate and move forward.
World wraps around - fall off a tile, reappear other side
Can't move through walls - just butt up against them

Answer needs row, column, facing
Rows start from 1 at the top and count downward; 
Columns start from 1 at the left and count rightward. 
Facing is 0 for right (>), 1 for down (v), 2 for left (<), and 3 for up (^). 

The final password is the sum of 1000 times the row, 4 times the column, and the facing.


In [1]:
testData = """        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5"""

import re
from pprint import pprint


VOID = ' '
TILE = '.'
WALL = '#'

RIGHT = 0
DOWN = 1
LEFT = 2
UP = 3

DIRECTION_VECTOR = {0:(0,1), 1:(1,0), 2:(0,-1), 3:(-1,0)}


class Maze:
    def __init__(self,input:str):
        self.maze = [] #let's use Row Column referencing
        self.height = 0
        self.width = 0
        self.instructions = []
        self.position = (0,0)
        self.direction = RIGHT

        mazeParseComplete = False
        for l in input.splitlines():
            if l == "":
                mazeParseComplete = True
            elif mazeParseComplete:
                self.instructions = re.split('(\d+)',l) #weirdly contains empty string at front and back... deal with this later
            else:
                row = []
                if self.width > 0:
                    l = l.ljust(self.width, ' ') #janky input file isn't fully padded
                else:
                    self.width = len(l)
                self.maze.append(list(l))
        self.height = len(self.maze)
        #self.width = len(self.maze[0])
        print('Maze initialised with H='+str(self.height)+' W='+str(self.width))

    
    def cubeFace(self, position:tuple[int,int])->int:
        #return which cube face current position maps onto.
        #going to hard code a load of part 2 stuff
        r, c = position
        if 0 <= r < 50 and 50 <= c < 100:
            return 1
        elif 0 <= r < 50 and 100 <= c < 150:
            return 2
        elif 50 <= r < 100 and 50 <= c < 100:
            return 3
        elif 100 <= r < 150 and 0 <= c < 50:
            return 4
        elif 100 <= r < 150 and 50 <= c < 100:
            return 5
        elif 150 <= r < 200 and 0 <= c < 50:
            return 6
        else:
            #off the cube
            return 0

    def updateNextPositionAndDirection(self)->bool:
        #returns false is no further movement in this direction possible (i.e. hit a wall)
        currentFace = self.cubeFace(self.position)
        moveVector = DIRECTION_VECTOR[self.direction]
        r, c = self.position
        nextPos = (self.position[0] + moveVector[0], self.position[1] + moveVector[1])
        nextFace = self.cubeFace(nextPos)
        nextDirection = self.direction
        if (currentFace == self.cubeFace(nextPos) 
            or (currentFace==1 and (nextFace==2 or nextFace==3))
            or (currentFace==2 and nextFace==1)
            or (currentFace==3 and (nextFace==1 or nextFace==5))
            or (currentFace==4 and (nextFace==5 or nextFace==6))
            or (currentFace==5 and (nextFace==3 or nextFace==4))
            or (currentFace==6 and nextFace==4)
            ):
            #movement in face, or across standard boundary, so no wrap or rotation required
            nextCell = self.maze[nextPos[0]][nextPos[1]]

        else:
            #need to world wrap
            #TODO: all the logic
            if currentFace==1 and self.direction==UP:
                nextDirection = RIGHT
                nextPos = (c-50+150,0)
            elif currentFace==1 and self.direction==LEFT:
                nextDirection = RIGHT
                nextPos = (149-r ,0)
            elif currentFace==2 and self.direction==UP:
                nextDirection = UP
                nextPos = (199 ,c-100)        
            elif currentFace==2 and self.direction==RIGHT:
                nextDirection = LEFT
                nextPos = (149-r ,99)   
            elif currentFace==2 and self.direction==DOWN:
                nextDirection = LEFT
                nextPos = (c-100+50,99)
            elif currentFace==3 and self.direction==LEFT:
                nextDirection = DOWN
                nextPos = (100, r-50)   
            elif currentFace==3 and self.direction==RIGHT:
                nextDirection = UP
                nextPos = (49, r-50+100)
            elif currentFace==4 and self.direction==UP:
                nextDirection = RIGHT
                nextPos = (c+50 ,50)   
            elif currentFace==4 and self.direction==LEFT:
                nextDirection = RIGHT
                nextPos = (49-(r-100), 50) 
            elif currentFace==5 and self.direction==RIGHT:
                nextDirection = LEFT
                nextPos = (49-(r-100) ,149)   
            elif currentFace==5 and self.direction==DOWN:
                nextDirection = LEFT
                nextPos = ((c-50)+150 ,49)
            elif currentFace==6 and self.direction==LEFT:
                nextDirection = DOWN
                nextPos = (0 ,r-150+50)   
            elif currentFace==6 and self.direction==RIGHT:
                nextDirection = UP
                nextPos = (149, r-150+50) 
            elif currentFace==6 and self.direction==DOWN:
                nextDirection = DOWN 
                nextPos = (0,c+100) 
            nextCell = self.maze[nextPos[0]][nextPos[1]]

        if nextCell == TILE:
            self.markPath()
            self.position = nextPos
            self.direction = nextDirection
            return True
        elif nextCell == WALL:
            return False
        else:
            raise Exception('falled off the cube')
       
    def setToStartPosition(self):
        #You begin the path in the leftmost open tile of the top row of tiles. 
        #Initially, you are facing to the right 
        for c in range(self.width):
            cell = self.maze[0][c]
            if cell == TILE:
                self.position = (0,c)
                break
        else:
            raise Exception('Tile not found in first row')


    def runTheMaze(self)->int:
        self.setToStartPosition()
        for instruction in self.instructions:
            #Need to determine if length or turn direction, and call appropriate function
            #print('Instruction: ' + instruction)
            if instruction == '':
                pass #skip
            elif instruction == 'L' or instruction == 'R':
                self.turn(instruction)
            else:
                self.walk(int(instruction))
                #print(self)
        #The final password is the sum of 1000 times the row, 4 times the column, and the facing.
        print('Final position:'+str(self.position)+' facing direction:'+str(self.direction))
        return 1000*(self.position[0]+1) + 4*(self.position[1]+1) + self.direction


    def walk(self,length:int):
        #walk step wise, until out of length, or hit a wall. World-wrap on the tiles.
        canMove = True
        while length > 0 and canMove:
            canMove = self.updateNextPositionAndDirection()
            length -= 1

    def turn(self,direction:str):
        if direction=='L':
            self.direction = (self.direction - 1) % 4
        elif direction=='R':
            self.direction = (self.direction + 1) % 4
        else:
            raise Exception('Unknown direction: '+direction)

    def markPath(self):
        #self.maze[self.position[0]][self.position[1]] = self.direction
        pass
    
    def __str__(self):
        output = ""
        for r in self.maze:
            for c in r:
                if isinstance(c, int):
                    output += str(c)
                else:
                    output += c
            output +=  '\n'
        return output

#tests
# tm = Maze(testData)
# print(tm.instructions)
# #print(tm)
# print(tm.runTheMaze())

puzzleinput = open('day22input.txt').read()
m = Maze(puzzleinput)
print(m.runTheMaze())


Maze initialised with H=200 W=150
Final position:(152, 49) facing direction:3
153203
